Добрый день! 
Задание по итогам третьего вебинара. 


Необходимо скачать и подготовить датасет https://www.kaggle.com/olekslu/makeup-lips-segmentation-28k-samples/notebooks   
    для обучения модели на губы  
И затем обучить Модель на выбор из segmentation_models_pytorch